In [194]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import io
import dvc.api
import seaborn as sns
import mlflow

In [164]:
# Importing Pandas an Numpy Libraries to use on manipulating our Data
import pandas as pd
import numpy as np

# To Preproccesing our data
from sklearn.preprocessing import LabelEncoder

# To fill missing values
from sklearn.impute import SimpleImputer

# To Split our train data
from sklearn.model_selection import train_test_split

# To Visualize Data
import matplotlib.pyplot as plt
import seaborn as sns

# To Train our data
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB, GaussianNB

# To evaluate end result we have
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score


In [165]:
#data = dvc.api.read(path='test_store.csv',repo='https://github.com/MohammedEsamaldin/Pharmaceutical-Sales-Prediction-across-multiple-stores.git')
df = pd.read_csv('../data/train_store.csv')

In [166]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,"('Sales', 'Customers')"
0,1,5,2015-07-31,5263.0,555.0,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,NaN
1,2,5,2015-07-31,6064.0,625.0,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",NaN
2,3,5,2015-07-31,8314.0,821.0,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NaN
3,4,5,2015-07-31,13995.0,1485.0,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,0.0,0,NaN
4,5,5,2015-07-31,4822.0,559.0,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0,NaN


In [167]:
#df.set_index('Date', inplace=True)

In [168]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,"('Sales', 'Customers')"
0,1,5,2015-07-31,5263.0,555.0,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,NaN
1,2,5,2015-07-31,6064.0,625.0,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",NaN
2,3,5,2015-07-31,8314.0,821.0,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",NaN
3,4,5,2015-07-31,13995.0,1485.0,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,0.0,0,NaN
4,5,5,2015-07-31,4822.0,559.0,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0,NaN


In [169]:
df['StateHoliday'].value_counts()

0    592943
0    393216
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

In [170]:
df['StateHoliday'] = df['StateHoliday'].values.astype(str)
df['StateHoliday'].replace({'0':'No Holiday','a':'Public Holiday','b':'Easter Holiday','c':'Christmas Holiday'},inplace=True)

In [171]:
def convert_to_datetime(df,cols):
        """
        Changing the columns into date time format
        """
        for col in cols:
            df[col] = pd.to_datetime(df[col])

In [172]:
convert_to_datetime(df, ['Date'])

In [173]:

df['Year'] = df['Date'].apply(lambda x: x.year)
df['Month'] = df['Date'].apply(lambda x: x.month)
df['DayOfMonth'] = df['Date'].apply(lambda x: x.day)
df['Weekday'] = df['DayOfWeek'].apply(lambda x: 0 if (x in [6, 7]) else 1) 

In [174]:
#
df = df.sort_values(by = 'Date')
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,"('Sales', 'Customers')",Year,Month,DayOfMonth,Weekday
1017208,1115,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,d,...,2010.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",NaN,2013,1,1,1
1016473,379,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,d,...,2010.0,0,0.0,0.0,0,NaN,2013,1,1,1
1016472,378,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,2012.0,0,0.0,0.0,0,NaN,2013,1,1,1
1016471,377,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,2010.0,1,18.0,2010.0,"Feb,May,Aug,Nov",NaN,2013,1,1,1
1016470,376,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,2012.0,0,0.0,0.0,0,NaN,2013,1,1,1


In [175]:
# Creating a new column

df.loc[df['DayOfMonth'] >= 20,'DayStatus'] = 'End of Month' 
df.loc[(df['DayOfMonth'] < 20) & (df['DayOfMonth'] > 10),'DayStatus'] = 'Mid of Month' 
df.loc[df['DayOfMonth'] < 10,'DayStatus'] = 'Begining of Month' 

In [176]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,"('Sales', 'Customers')",Year,Month,DayOfMonth,Weekday,DayStatus
1017208,1115,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,d,...,1,22.0,2012.0,"Mar,Jun,Sept,Dec",NaN,2013,1,1,1,Begining of Month
1016473,379,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,d,...,0,0.0,0.0,0,NaN,2013,1,1,1,Begining of Month
1016472,378,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,0,0.0,0.0,0,NaN,2013,1,1,1,Begining of Month
1016471,377,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,1,18.0,2010.0,"Feb,May,Aug,Nov",NaN,2013,1,1,1,Begining of Month
1016470,376,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,0,0.0,0.0,0,NaN,2013,1,1,1,Begining of Month


In [177]:
holidays = ["30.12.2014", "02.01.2015", "10.01.2015"]
holidays = pd.to_datetime(holidays, dayfirst=True)

df1 = pd.DataFrame({'date1':holidays})

df = pd.merge_asof(df, df1, left_on='Date', right_on='date1', direction='forward')
df = pd.merge_asof(df, df1, left_on='Date', right_on='date1')

df['until'] = df.pop('date1_x').sub(df['Date']).dt.days
df['since'] = df['Date'].sub(df.pop('date1_y')).dt.days
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,Promo2SinceYear,PromoInterval,"('Sales', 'Customers')",Year,Month,DayOfMonth,Weekday,DayStatus,until,since
0,1115,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,d,...,2012.0,"Mar,Jun,Sept,Dec",NaN,2013,1,1,1,Begining of Month,728.0,NaN
1,379,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,d,...,0.0,0,NaN,2013,1,1,1,Begining of Month,728.0,NaN
2,378,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,0.0,0,NaN,2013,1,1,1,Begining of Month,728.0,NaN
3,377,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,2010.0,"Feb,May,Aug,Nov",NaN,2013,1,1,1,Begining of Month,728.0,NaN
4,376,2,2013-01-01,0.0,0.0,0,0,Public Holiday,1,a,...,0.0,0,NaN,2013,1,1,1,Begining of Month,728.0,NaN


In [178]:

df.drop("('Sales', 'Customers')", axis=1, inplace=True)

In [179]:
numerical_column = df.select_dtypes(exclude="object").columns.tolist()
categorical_column = df.select_dtypes(include="object").columns.tolist()
print("Numerical Columns:", numerical_column)
print("****************")
print("Categorical Columns:", categorical_column)


Numerical Columns: ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'DayOfMonth', 'Weekday', 'until', 'since']
****************
Categorical Columns: ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval', 'DayStatus']


In [180]:

# Get column names have less than 10 more than 2 unique values
to_one_hot_encoding = []

# Get Categorical Column names thoose are not in "to_one_hot_encoding"
to_label_encoding = [col for col in categorical_column if df[col].nunique() <= 10 and df[col].nunique() > 2]
print("To Label Encoding:", to_label_encoding)


To Label Encoding: ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval', 'DayStatus']


In [181]:


label_encoded_columns = []
# For loop for each columns
for col in to_label_encoding:
    # We define new label encoder to each new column
    le = LabelEncoder()
    # Encode our data and create new Dataframe of it, 
    # notice that we gave column name in "columns" arguments
    column_dataframe = pd.DataFrame(le.fit_transform(df[col]), columns=[col] )
    # and add new DataFrame to "label_encoded_columns" list
    label_encoded_columns.append(column_dataframe)

# Merge all data frames
label_encoded_columns = pd.concat(label_encoded_columns, axis=1)
label_encoded_columns
'''
# We will use built in pandas function "get_dummies()" to simply to encode "to_one_hot_encoding" columns
one_hot_encoded_columns = pd.get_dummies(df[to_one_hot_encoding])
one_hot_encoded_columns
'''

'\n# We will use built in pandas function "get_dummies()" to simply to encode "to_one_hot_encoding" columns\none_hot_encoded_columns = pd.get_dummies(df[to_one_hot_encoding])\none_hot_encoded_columns\n'

In [182]:
df.dtypes

Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                               float64
Customers                           float64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
Year                                  int64
Month                                 int64
DayOfMonth                            int64
Weekday                               int64
DayStatus                       

In [183]:

# Copy our DataFrame to X variable
X = df.copy()

# Droping Categorical Columns,
# "inplace" means replace our data with new one
# Don't forget to "axis=1"
X.drop(categorical_column, axis=1, inplace=True)

# Merge DataFrames
X = pd.concat([X, label_encoded_columns], axis=1)
print("All columns:", X.columns.tolist())
X.set_index('Date',inplace=True)

All columns: ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'DayOfMonth', 'Weekday', 'until', 'since', 'StateHoliday', 'StoreType', 'Assortment', 'PromoInterval', 'DayStatus']


In [184]:
y = df['Sales']

# Droping "class" from X
X.drop(['Sales','until','since' ], axis=1, inplace=True)
X

,Store,DayOfWeek,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,...,Promo2SinceYear,Year,Month,DayOfMonth,Weekday,StateHoliday,StoreType,Assortment,PromoInterval,DayStatus
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1115,2,0.0,0,0,1,5350.0,8.0,2010.0,1,...,2012.0,2013,1,1,1,3,3,2,3,0
2013-01-01,379,2,0.0,0,0,1,6630.0,8.0,2010.0,0,...,0.0,2013,1,1,1,3,3,0,0,0
2013-01-01,378,2,0.0,0,0,1,2140.0,8.0,2012.0,0,...,0.0,2013,1,1,1,3,0,2,0,0
2013-01-01,377,2,0.0,0,0,1,100.0,6.0,2010.0,1,...,2010.0,2013,1,1,1,3,0,2,1,0
2013-01-01,376,2,0.0,0,0,1,160.0,8.0,2012.0,0,...,0.0,2013,1,1,1,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-07-31,746,5,638.0,1,1,1,4330.0,2.0,2011.0,1,...,2011.0,2015,7,31,1,2,3,2,3,1
2015-07-31,747,5,826.0,1,1,1,45740.0,8.0,2008.0,0,...,0.0,2015,7,31,1,2,2,2,0,1
2015-07-31,748,5,578.0,1,1,1,2380.0,3.0,2010.0,1,...,2011.0,2015,7,31,1,2,3,0,2,1


In [185]:
# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.7)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.333)

Predicted: [    0.  4117.  7160.  3864. 10522.  6130.  4016.     0.  9005.  8578.]
Actual: 232655        0.0
709991     4207.0
88596      7041.0
80484      3792.0
175436    10076.0
127967     6220.0
582249     3950.0
380277        0.0
457384     8498.0
190069     8634.0
Name: Sales, dtype: float64


In [187]:
# Metrics for Evaluation of model Accuracy and F1-score
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [202]:
with mlflow.start_run():
    # Fit regression model
    regr = DecisionTreeRegressor(random_state=10)
    regr.fit(X_train, y_train)
    # Predict
    result = regr.predict(X_test)
    print("Predicted:", result[0:10])
    print("Actual:", y_test[0:10])
    # for classification we use accuracy and F1 score
    accuracy = round(regr.score(X_test,y_test),2)
    print('accuracy:',accuracy)
    #print(f1_score(y_test,result))
    
    # for regression we use R2 score and MAE(mean absolute error)
    # all other steps will be same as classification as shown above

    print('mean absolute error:',round(mean_absolute_error(y_test,result),2))
    mlflow.log_param('random_state',10)
    mlflow.log_metric('accuracy',accuracy)
    mlflow.sklearn.log_model(regr,'model')
    modelpath = '../models'
    mlflow.sklearn.save_model(regr,modelpath)
    mlflow.set_experiment("my-experiment")


Predicted: [    0.  4566.  7300.  3960. 10522.  6054.  3935.     0.  9005.  8308.]
Actual: 232655        0.0
709991     4207.0
88596      7041.0
80484      3792.0
175436    10076.0
127967     6220.0
582249     3950.0
380277        0.0
457384     8498.0
190069     8634.0
Name: Sales, dtype: float64
accuracy: 0.98
mean absolute error: 355.02


Traceback (most recent call last):
  File "c:\Users\mohammed\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 270, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "c:\Users\mohammed\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 394, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\Users\mohammed\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 1049, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\Users\mohammed\anaconda3\lib\site-packages\mlflow\store\tracking\file_store.py", line 1042, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\Users\mohammed\anaconda3\lib\site-packages\mlflow\utils\file_utils.py", line 181, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file 'C:\

MlflowException: Invalid experiment ID: 'decision_tree2.pkl'

In [189]:
db = df
db['ds']= df['Date'].values
db['y'] = df['Sales'].values
db.drop(columns=['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'DayOfMonth', 'Weekday', 'until', 'since', 'StateHoliday', 'StoreType', 'Assortment', 'PromoInterval', 'DayStatus'],inplace=True)

In [190]:
#db.set_index('ds',inplace= True )
db.head()

,ds,y
0,2013-01-01,0.0
1,2013-01-01,0.0
2,2013-01-01,0.0
3,2013-01-01,0.0
4,2013-01-01,0.0


In [191]:
from prophet import Prophet


In [192]:
m = Prophet()
m.fit(db)


17:36:22 - cmdstanpy - INFO - Chain [1] start processing
17:37:01 - cmdstanpy - INFO - Chain [1] done processing
17:37:01 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
17:39:47 - cmdstanpy - INFO - Chain [1] start processing
17:40:07 - cmdstanpy - INFO - Chain [1] done processing
17:40:07 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


RuntimeError: Error during optimization: 

In [205]:
df.to_csv('../data/preproess_ml.csv')